In [1]:
###IMPORTS

from __future__ import division

import ladybug.epw as epw

import collections
import array
import math

import rhinoinside
rhinoinside.load()
import System
import Rhino

try:
    import Rhino.UI as rui
except ImportError as e:
    raise ImportError("Failed to import Rhino.\n{}".format(e))

try:
    from System import Object
    from System.Windows import Forms
    from System import Environment
    import System.Security.Principal as sec
    import System.Threading.Tasks as tasks
except ImportError:
    print("Failed to import System.")




IMPORT_EPW

In [3]:

_epw_file = "FRA_AC_Agen-La.Garenne.AP.075240_TMYx.epw"

epw_data = epw.EPW(_epw_file)
location = epw_data.location
dry_bulb_temperature = epw_data.dry_bulb_temperature
dew_point_temperature = epw_data.dew_point_temperature
relative_humidity = epw_data.relative_humidity
wind_speed = epw_data.wind_speed
wind_direction = epw_data.wind_direction
direct_normal_rad = epw_data.direct_normal_radiation
diffuse_horizontal_rad = epw_data.diffuse_horizontal_radiation
global_horizontal_rad = epw_data.global_horizontal_radiation
horizontal_infrared_rad = epw_data.horizontal_infrared_radiation_intensity
direct_normal_ill = epw_data.direct_normal_illuminance
diffuse_horizontal_ill = epw_data.diffuse_horizontal_illuminance
global_horizontal_ill = epw_data.global_horizontal_illuminance
total_sky_cover = epw_data.total_sky_cover
barometric_pressure = epw_data.atmospheric_station_pressure
model_year = epw_data.years
g_temp = epw_data.monthly_ground_temperature
ground_temperature = [g_temp[key] for key in sorted(g_temp.keys())]


#####
ORIENTATION

In [17]:

ptOri = Rhino.Geometry.Point3d(0,0,0)
pt1 = Rhino.Geometry.Point3d(1,0,0)
pt2 = Rhino.Geometry.Point3d(0,1,0)

plan1 = Rhino.Geometry.Plane(ptOri,pt1,pt2)